<a href="https://colab.research.google.com/github/AForbis/Bootcamp_Final_Project/blob/datasourcing/final_project_segment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import requests
import io
import csv

In [2]:
#eventually we will replace this with a database
url = 'SeasonData/AllNBA_Unpivoted.xlsx'
#load in All NBA teams, drop the league column (we are not including the ABA, I already deleted all that from the file)
All_NBA_Unpivoted_df = pd.DataFrame(pd.read_excel(url)).drop(['Lg'],axis=1)
#rename the players column
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Value':'PlayerName'})
#create column with the start year
All_NBA_Unpivoted_df['Season_Start_Year'] = All_NBA_Unpivoted_df['Season'].apply(lambda row: row[0:4])
All_NBA_Unpivoted_df['Season_Start_Year'] = pd.to_numeric(All_NBA_Unpivoted_df['Season_Start_Year'])
#All_NBA_Unpivoted_df.head(10)

,Season,Tm,Position,PlayerName,Season_Start_Year
0,2019-20,1st,C,Anthony Davis,2019
1,2019-20,1st,F,Giannis Antetokounmpo,2019
2,2019-20,1st,F,LeBron James,2019
3,2019-20,1st,G,James Harden,2019
4,2019-20,1st,G,Luka Dončić,2019
5,2019-20,2nd,C,Nikola Jokić,2019
6,2019-20,2nd,F,Kawhi Leonard,2019
7,2019-20,2nd,F,Pascal Siakam,2019
8,2019-20,2nd,G,Damian Lillard,2019
9,2019-20,2nd,G,Chris Paul,2019


In [3]:
#cleaning up the names with crazy characters
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName'].apply(lambda row: row.replace("ć","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("č","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("ž","z"))

#make dummies for position and team (1st, 2nd, 3rd)
All_NBA_Unpivoted_df = pd.get_dummies(All_NBA_Unpivoted_df,columns=['Position','Tm']).drop(['Season','PlayerName'],axis=1)
#rename the columns (cosmetic)
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_C':'AllNBA_Center'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_F':'AllNBA_Forward'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_G':'AllNBA_Guard'})
#All_NBA_Unpivoted_df.head(10)

,Season_Start_Year,PlayerName_Formatted,AllNBA_Center,AllNBA_Forward,AllNBA_Guard,Tm_1st,Tm_2nd,Tm_3rd
0,2019,Anthony Davis,1,0,0,1,0,0
1,2019,Giannis Antetokounmpo,0,1,0,1,0,0
2,2019,LeBron James,0,1,0,1,0,0
3,2019,James Harden,0,0,1,1,0,0
4,2019,Luka Doncic,0,0,1,1,0,0
5,2019,Nikola Jokic,1,0,0,0,1,0
6,2019,Kawhi Leonard,0,1,0,0,1,0
7,2019,Pascal Siakam,0,1,0,0,1,0
8,2019,Damian Lillard,0,0,1,0,1,0
9,2019,Chris Paul,0,0,1,0,1,0


In [4]:
#get list of all of the players that have made an all nba team
AllNBAPlayersList = All_NBA_Unpivoted_df['PlayerName_Formatted'].unique()
#AllNBAPlayersList

In [5]:
url = 'SeasonData/Seasons_Stats.xlsx'
#load in season stats
season_stats_df = pd.DataFrame(pd.read_excel(url))
season_stats_df['Player'] = season_stats_df['Player'].apply(lambda row: row.replace("*",""))
season_stats_df=season_stats_df.rename(columns = {'Year':'Year_season_end'})
season_stats_df['Year_season_start'] = (season_stats_df['Year_season_end'] - 1)
#season_stats_df.head()
#season_stats_df.to_csv('ML_DF.csv')

,Year_season_end,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year_season_start
0,2017,A.J. Hammons,C,24,DAL,22,0.0,163,8.4,0.472,...,8,28,36,4,1,13,10,21,48,2016
1,2017,Aaron Brooks,PG,32,IND,65,0.0,894,9.5,0.507,...,18,51,69,125,25,9,66,93,322,2016
2,2017,Aaron Gordon,SF,21,ORL,80,72.0,2298,14.4,0.530,...,116,289,405,150,64,40,89,172,1019,2016
3,2017,Aaron Harrison,SG,22,CHO,5,0.0,17,-2.2,0.102,...,0,3,3,3,0,0,0,2,1,2016
4,2017,Adreian Payne,PF,25,MIN,18,0.0,135,14.4,0.505,...,9,24,33,7,8,7,8,32,63,2016


In [6]:
# get list of every nba player
AllPlayersList = season_stats_df.Player.unique()
AllPlayersList_df = pd.DataFrame(AllPlayersList)
#AllPlayersList_df.to_csv('listofplayers_test.csv')

In [7]:
#check and make sure every all-nba player shows up in the season data dataset
#players who weren't in the league in 2017 will show up here, that's okay!
#I cleaned up a bunch of names in the excel
for player in AllNBAPlayersList:
    if player not in AllPlayersList:   
        print(player)

Luka Doncic
Jayson Tatum
Ben Simmons


In [8]:
#merge allNBA status into season stats
Season_Stats_ML_DF = pd.merge(season_stats_df,All_NBA_Unpivoted_df,how='left',left_on = ['Player','Year_season_start'], right_on = ['PlayerName_Formatted','Season_Start_Year'])

In [9]:
# get rid of duplicate cols
Season_Stats_ML_DF = Season_Stats_ML_DF.drop(['Season_Start_Year','PlayerName_Formatted'],axis=1)

In [10]:
# fill in zeros for players that arent all nba
Season_Stats_ML_DF['AllNBA_Center'] = Season_Stats_ML_DF['AllNBA_Center'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Forward'] = Season_Stats_ML_DF['AllNBA_Forward'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Guard'] = Season_Stats_ML_DF['AllNBA_Guard'].fillna(int(0))
Season_Stats_ML_DF['Tm_1st'] = Season_Stats_ML_DF['Tm_1st'].fillna(int(0))
Season_Stats_ML_DF['Tm_2nd'] = Season_Stats_ML_DF['Tm_2nd'].fillna(int(0))
Season_Stats_ML_DF['Tm_3rd'] = Season_Stats_ML_DF['Tm_3rd'].fillna(int(0))


,Year_season_end,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,TOV,PF,PTS,Year_season_start,AllNBA_Center,AllNBA_Forward,AllNBA_Guard,Tm_1st,Tm_2nd,Tm_3rd
0,2017,A.J. Hammons,C,24,DAL,22,0.0,163,8.4,0.472,...,10,21,48,2016,0.0,0.0,0.0,0.0,0.0,0.0
1,2017,Aaron Brooks,PG,32,IND,65,0.0,894,9.5,0.507,...,66,93,322,2016,0.0,0.0,0.0,0.0,0.0,0.0
2,2017,Aaron Gordon,SF,21,ORL,80,72.0,2298,14.4,0.530,...,89,172,1019,2016,0.0,0.0,0.0,0.0,0.0,0.0
3,2017,Aaron Harrison,SG,22,CHO,5,0.0,17,-2.2,0.102,...,0,2,1,2016,0.0,0.0,0.0,0.0,0.0,0.0
4,2017,Adreian Payne,PF,25,MIN,18,0.0,135,14.4,0.505,...,8,32,63,2016,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
#get list of positions
Positions_List = Season_Stats_ML_DF['Pos'].unique()


array(['C', 'PG', 'SF', 'SG', 'PF', 'PF-C', 'SG-SF', 'PG-SG', 'SG-PG',
       'PF-SF', 'SF-SG', 'SF-PF', 'C-PF', 'SG-PF', 'C-SF', 'PG-SF'],
      dtype=object)

In [12]:
# create functions to determine if player is a guard forward and/or center
def is_forward(pos):
    possible_pos = [ 'SF',  'PF', 'PF-C', 'SG-SF', 'PF-SF', 'SF-SG', 'SF-PF', 'C-PF', 'SG-PF', 'C-SF', 'PG-SF']
    if pos in possible_pos: isF = 1
    else: isF = 0
    return isF

def is_guard(pos):
    possible_pos = ['PG','SG','SG-SF','PG-SG','SG-PG','SF-SG','SG-PF','PG-SF']
    if pos in possible_pos: isG = 1
    else: isG = 0
    return isG

def is_center(pos):
    possible_pos = ['C','PF-C','C-PF','C-SF']
    if pos in possible_pos: isC = 1
    else: isC = 0
    return isC

In [13]:
#Create dummy columns for G, F, C
Season_Stats_ML_DF['is_Guard'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_guard(pos) )
Season_Stats_ML_DF['is_Forward'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_forward(pos) )
Season_Stats_ML_DF['is_Center'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_center(pos) )


,Year_season_end,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,Year_season_start,AllNBA_Center,AllNBA_Forward,AllNBA_Guard,Tm_1st,Tm_2nd,Tm_3rd,is_Guard,is_Forward,is_Center
0,2017,A.J. Hammons,C,24,DAL,22,0.0,163,8.4,0.472,...,2016,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
1,2017,Aaron Brooks,PG,32,IND,65,0.0,894,9.5,0.507,...,2016,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0
2,2017,Aaron Gordon,SF,21,ORL,80,72.0,2298,14.4,0.530,...,2016,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
3,2017,Aaron Harrison,SG,22,CHO,5,0.0,17,-2.2,0.102,...,2016,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0
4,2017,Adreian Payne,PF,25,MIN,18,0.0,135,14.4,0.505,...,2016,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0


In [14]:
#Drop the Pos Column since we have dummy columns
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Pos'],axis=1)


In [15]:
#make dummies for team
Season_Stats_ML_DF = pd.get_dummies(Season_Stats_ML_DF,columns=['Tm','Year_season_start'])


,Year_season_end,Player,Age,G,GS,MP,PER,TS%,3PAr,FTr,...,Year_season_start_2007,Year_season_start_2008,Year_season_start_2009,Year_season_start_2010,Year_season_start_2011,Year_season_start_2012,Year_season_start_2013,Year_season_start_2014,Year_season_start_2015,Year_season_start_2016
0,2017,A.J. Hammons,24,22,0.0,163,8.4,0.472,0.238,0.476,...,0,0,0,0,0,0,0,0,0,1
1,2017,Aaron Brooks,32,65,0.0,894,9.5,0.507,0.427,0.133,...,0,0,0,0,0,0,0,0,0,1
2,2017,Aaron Gordon,21,80,72.0,2298,14.4,0.530,0.309,0.251,...,0,0,0,0,0,0,0,0,0,1
3,2017,Aaron Harrison,22,5,0.0,17,-2.2,0.102,0.500,0.500,...,0,0,0,0,0,0,0,0,0,1
4,2017,Adreian Payne,25,18,0.0,135,14.4,0.505,0.278,0.352,...,0,0,0,0,0,0,0,0,0,1


In [17]:
# drop year end and player name
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Year_season_end','Player'],axis=1)
Season_Stats_ML_DF.head()

,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,...,Year_season_start_2007,Year_season_start_2008,Year_season_start_2009,Year_season_start_2010,Year_season_start_2011,Year_season_start_2012,Year_season_start_2013,Year_season_start_2014,Year_season_start_2015,Year_season_start_2016
0,24,22,0.0,163,8.4,0.472,0.238,0.476,5.4,20.9,...,0,0,0,0,0,0,0,0,0,1
1,32,65,0.0,894,9.5,0.507,0.427,0.133,2.3,6.3,...,0,0,0,0,0,0,0,0,0,1
2,21,80,72.0,2298,14.4,0.530,0.309,0.251,5.3,14.1,...,0,0,0,0,0,0,0,0,0,1
3,22,5,0.0,17,-2.2,0.102,0.500,0.500,0.0,19.5,...,0,0,0,0,0,0,0,0,0,1
4,25,18,0.0,135,14.4,0.505,0.278,0.352,7.7,21.0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
#Season_Stats_ML_DF.to_csv('ML_DF.csv')